# THE 8-PUZZLE

## STANDARD FORMULATION

Including path to previous directory in built-in variable `sys.path`:

In [1]:
import sys

sys.path.append('../')

Importing the standard formulation:

In [2]:
import numpy as np
from collections import deque
from puzzle import formulations

pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


Class with a search problem:

In [3]:
class Problem:
    def __init__(self, initial, objective, actions, result):
        self.initial = initial
        self.objective = objective
        self.actions = actions
        self.result = result  # defines the transition model

Class with a tree node:

In [4]:
class Node:
    def __init__(self, state, cost, parent, action):
        self.state = state
        self.cost = cost
        self.parent = parent
        self.action = action

    def __str__(self):  # representing the node
        return '\n\n'.join(['  '.join(map(str, grid)) for grid in self.state[0:2]]) + \
               '  >>>  ' + str(self.cost) + '\n\n' + \
               '\n\n'.join(['  '.join(map(str, grid)) for grid in self.state[2:]])

    @classmethod
    def child(cls, problem, parent, action):
        state = problem.result(parent.state, action)
    
        return cls(state, parent.cost + 1, parent, action)

    @property
    def hash(self):
        return np.array2string(self.state)

    @property
    def solution(self):
        node = self;
        solution = []

        while node:
            solution.append(node)
            node = node.parent

        solution.reverse()

        return solution

Defining the constants and creating new instances of the problem:

In [5]:
N = 3
OBJ_GRID = np.arange(N**2, dtype='int8').reshape((N, N))


grid1 = np.array([[3, 2, 7],
                  [4, 8, 1],
                  [6, 5, 0]])

grid2 = np.array([[3, 2, 7],
                  [4, 8, 0],
                  [6, 5, 1]])

grid3 = np.array([[3, 2, 0],
                  [4, 8, 7],
                  [6, 5, 1]])

grid4 = np.array([[2, 6, 1],
                  [7, 5, 0],
                  [8, 3, 4]])

puzzle_12_steps = Problem(Node(grid1, 0, None, None), formulations.won_comp,
                          formulations.available_moves_search, formulations.move_grid)

puzzle_13_steps = Problem(Node(grid2, 0, None, None), formulations.won_comp,
                          formulations.available_moves_search, formulations.move_grid)

puzzle_14_steps = Problem(Node(grid3, 0, None, None), formulations.won_comp,
                          formulations.available_moves_search, formulations.move_grid)

puzzle_23_steps = Problem(Node(grid4, 0, None, None), formulations.won_comp,
                          formulations.available_moves_search, formulations.move_grid)

print(puzzle_12_steps.initial)
print('=' * 7)
print(puzzle_13_steps.initial)
print('=' * 7)
print(puzzle_14_steps.initial)
print('=' * 7)
print(puzzle_23_steps.initial)

3  2  7

4  8  1  >>>  0

6  5  0
3  2  7

4  8  0  >>>  0

6  5  1
3  2  0

4  8  7  >>>  0

6  5  1
2  6  1

7  5  0  >>>  0

8  3  4


## BREADTH-FIRST SEARCH

Constants for current search status:

In [6]:
SEARCH_NOT_STARTED = 0
SEARCH_STARTED = 1
SEARCH_FAIL = 2
SEARCH_SUCCESS = 3

Class that will implement breadth-first search:

In [7]:
class BFS:
    def __init__(self, problem):
        self.problem   = problem
        self.explored  = set()
        self.frontier  = deque([])
        self.situation = SEARCH_NOT_STARTED
        self.solution  = None

    def step_search(self):
        # Empty border ends the search
        if not self.frontier:
            self.situation = SEARCH_FAIL
            return

        # Performing the search step
        node = self.frontier.popleft()

        if self.problem.objective(node.state, OBJ_GRID):
            self.solution = node.solution
            self.situation = SEARCH_SUCCESS
            return

        self.explored.add(node.hash)

        for action in self.problem.actions(node.state, node.action):
            child = Node.child(self.problem, node, action)
            
            if child not in self.frontier and not child.hash in self.explored:                
                self.frontier.append(child)

    def search(self):
        if self.situation == SEARCH_NOT_STARTED:
            self.frontier.append(self.problem.initial)
            self.situation = SEARCH_STARTED
        elif self.situation == SEARCH_FAIL:
            print("Search process failed!")
            return
        elif self.situation == SEARCH_SUCCESS:
            print("Solution already found!")
            return

        # Loop that performs breadth-first search
        while self.situation == SEARCH_STARTED:
            self.step_search()

        if self.situation == SEARCH_FAIL:
            print('Search process failed!')
        else:
            print('Solution found!')

    @property
    def show_solution(self):
        if self.situation == SEARCH_SUCCESS:
            return '\n\n'.join([node.__str__() for node in self.solution]) + \
                  f'\n\nCost: {self.solution[-1].cost}'
    
        return 'Solution still not found!'

    @property
    def show_frontier(self):
        return '#'*15 + '\n' + \
               '\n'.join([node.__str__() for node in self.frontier]) + \
               '\n' + '#'*15

Testing the efficiency of the breadth-first search:

In [8]:
%%time

bfs = BFS(puzzle_12_steps)
bfs.search()

Solution found!
CPU times: total: 281 ms
Wall time: 361 ms


In [9]:
%%time

bfs = BFS(puzzle_13_steps)
bfs.search()

Solution found!
CPU times: total: 375 ms
Wall time: 823 ms


In [10]:
%%time

bfs = BFS(puzzle_14_steps)
bfs.search()

Solution found!
CPU times: total: 406 ms
Wall time: 1.37 s


Breadth-first search via function:

In [11]:
def bfs(problem):
    node = problem.initial
    
    if problem.objective(node.state, OBJ_GRID):
        return node.solution

    explored = set()
    frontier = deque([node])
    
    while True:
        if not frontier:
            print("Search process failed!")
            return None

        # Performing the search step
        node = frontier.popleft()
        
        explored.add(node.hash)
        
        for action in problem.actions(node.state, node.action):
            child = Node.child(problem, node, action)

            if child not in frontier and not child.hash in explored:
                if problem.objective(child.state, OBJ_GRID):
                    print("Solution found!")
                    return child.solution

                frontier.append(child)

def show_solution(solution):
    print('\n\n'.join([node.__str__() for node in solution]) + \
         f'\n\nCost: {solution[-1].cost}')

Checking the efficiency:

In [12]:
%%time

_ = bfs(puzzle_12_steps)

Solution found!
CPU times: total: 109 ms
Wall time: 216 ms


In [13]:
%%time

_ = bfs(puzzle_13_steps)

Solution found!
CPU times: total: 203 ms
Wall time: 525 ms


In [14]:
%%time

_ = bfs(puzzle_14_steps)

Solution found!
CPU times: total: 375 ms
Wall time: 830 ms


## BACKTRACKING SEARCH

In [15]:
reverse_state = {'l': 'r',
                 'r': 'l',
                 'd': 'u',
                 'u': 'd'}

def backtracking(problem, limit):
    solution = np.array([None for i in range(limit+1)])
    
    size = 0
    solution[size] = (problem.initial.state,
                      problem.actions(problem.initial.state, None))
    
    while not problem.objective(solution[size][0], OBJ_GRID):        
        if size == limit:
            size -= 1
            
            while not solution[size][1]:
                size -= 1
                
                if size == -1:
                    return None

        action = solution[size][1].pop()
        
        state = problem.result(solution[size][0], action)
        size += 1
        solution[size] = (state, problem.actions(state, action))
    
    return solution

Testing backtracking search:

In [16]:
%%time

solution = backtracking(puzzle_12_steps, 12)

len(solution)

CPU times: total: 0 ns
Wall time: 15.6 ms


13

In [17]:
%%time

solution = backtracking(puzzle_13_steps, 13)

len(solution)

CPU times: total: 15.6 ms
Wall time: 15.6 ms


14

In [18]:
%%time

solution = backtracking(puzzle_14_steps, 14)

len(solution)

CPU times: total: 0 ns
Wall time: 16 ms


15

In [19]:
%%time

solution = backtracking(puzzle_23_steps, 23)

len(solution)

CPU times: total: 13.2 s
Wall time: 27.9 s


24